<a href="https://colab.research.google.com/github/gerrygestario/Spam_SD.SMS.Paylater_Kelompok-1/blob/main/Spam_SD_sms_paylater_kelompok_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deteksi Spam SMS Paylater menggunakan teknik NLP berbasis TFIDF dengan Logistic Regression dan Support Vector Machine

#Import Library dan Data

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

# 1. Load Dataset Paylater (Data Spesifik)
df_paylater = pd.read_csv('dataset_spam_paylater.csv')

# 2. Load Dataset Spam Umum (Data Tambahan untuk memperkuat model)
# Dataset spam.csv biasanya menggunakan encoding latin-1
df_spam_general = pd.read_csv('spam.csv', encoding='latin-1')

# Menampilkan sampel data
print("Data Paylater:")
print(df_paylater.head())
print("\nData Spam Umum:")
print(df_spam_general.head())

Data Paylater:
  label                                               text
0  spam  Akun PayLater Anda akan diblokir jika tidak se...
1   ham                        Transaksi berhasil diproses
2   ham                        Kode OTP Anda adalah 123456
3  spam  Tagihan PayLater Anda sebesar Rp3.000.000 belu...
4   ham                        Transaksi berhasil diproses

Data Spam Umum:
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


#Penggabungan dan Pembersihan data

In [ ]:
# Membersihkan dataset Spam Umum
# Kita hanya butuh kolom v1 (label) dan v2 (text)
df_spam_general = df_spam_general[['v1', 'v2']]
df_spam_general.columns = ['label', 'text'] # Rename agar sama dengan df_paylater

# Menggabungkan kedua dataset
df_combined = pd.concat([df_spam_general, df_paylater], ignore_index=True)

print(f"Jumlah total data setelah digabung: {df_combined.shape[0]} baris")
print(df_combined['label'].value_counts())

# Fungsi Preprocessing Teks Sederhana
def clean_text(text):
    text = str(text).lower() # Ubah ke huruf kecil
    text = re.sub('\[.*?\]', '', text) # Hapus teks dalam kurung siku
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Hapus tanda baca
    text = re.sub('\w*\d\w*', '', text) # Hapus kata yang mengandung angka
    return text

# Terapkan preprocessing
df_combined['text_clean'] = df_combined['text'].apply(clean_text)

print("\nContoh data setelah preprocessing:")
print(df_combined[['text', 'text_clean']].head())

Jumlah total data setelah digabung: 6072 baris
label
ham     5075
spam     997
Name: count, dtype: int64

Contoh data setelah preprocessing:
                                                text  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   

                                          text_clean  
0  go until jurong point crazy available only in ...  
1                            ok lar joking wif u oni  
2  free entry in  a wkly comp to win fa cup final...  
3        u dun say so early hor u c already then say  
4  nah i dont think he goes to usf he lives aroun...  


<>:15: SyntaxWarning: invalid escape sequence '\['
<>:17: SyntaxWarning: invalid escape sequence '\w'
<>:15: SyntaxWarning: invalid escape sequence '\['
<>:17: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-3067836199.py:15: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text) # Hapus teks dalam kurung siku
/tmp/ipython-input-3067836199.py:17: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text) # Hapus kata yang mengandung angka


#TFIDF dan Split Data

In [ ]:
# Fitur (X) dan Target (y)
X = df_combined['text_clean']
y = df_combined['label']

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi TF-IDF
tfidf = TfidfVectorizer(max_features=3000) # Ambil 3000 kata terpenting

# Transformasi data
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("Shape X_train_tfidf:", X_train_tfidf.shape)
print("Shape X_test_tfidf:", X_test_tfidf.shape)

Shape X_train_tfidf: (4857, 3000)
Shape X_test_tfidf: (1215, 3000)


#Training Model ( Logistic Regression dan Support Vector Machine)

In [ ]:
# --- Model 1: Logistic Regression ---
lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)

# --- Model 2: Support Vector Machine (SVM) ---
svm_model = SVC(kernel='linear', probability=True) # Kernel linear biasanya bagus untuk teks
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)

# --- Evaluasi ---
print("=== Evaluasi Logistic Regression ===")
print("Akurasi:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

print("\n=== Evaluasi SVM ===")
print("Akurasi:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))

=== Evaluasi Logistic Regression ===
Akurasi: 0.9646090534979423
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1023
        spam       0.99      0.78      0.87       192

    accuracy                           0.96      1215
   macro avg       0.98      0.89      0.93      1215
weighted avg       0.97      0.96      0.96      1215


=== Evaluasi SVM ===
Akurasi: 0.9860082304526749
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1023
        spam       0.99      0.92      0.95       192

    accuracy                           0.99      1215
   macro avg       0.99      0.96      0.97      1215
weighted avg       0.99      0.99      0.99      1215



#Menyimpan Model

In [ ]:
# Simpan Vectorizer dan Model
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

with open('model_lr.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

with open('model_svm.pkl', 'wb') as f:
    pickle.dump(svm_model, f)

print("Model dan Vectorizer berhasil disimpan!")

Model dan Vectorizer berhasil disimpan!


#Membuat Aplikasi Streamlit (app.py)

In [26]:
%%writefile app.py
import streamlit as st
import pickle
import re
import string

# --- Fungsi Preprocessing ---
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# --- Load Model & Vectorizer ---
@st.cache_resource
def load_models():
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        vectorizer = pickle.load(f)
    with open('model_svm.pkl', 'rb') as f:
        model_svm = pickle.load(f)
    return vectorizer, model_svm

tfidf, model = load_models()

# --- Daftar Contoh Pesan ---
contoh_pesan = {
    "--- Pilih Contoh Pesan ---": "",
    "Tagihan Jatuh Tempo": "Peringatan! Tagihan PayLater Anda senilai Rp2.500.000 jatuh tempo hari ini. Segera bayar ke Virtual Account 8899001122 agar tidak terkena denda harian.",
    "Akun Diblokir": "Akun PayLater Anda ditangguhkan sementara karena aktivitas mencurigakan. Segera verifikasi data diri Anda di link: bit.ly/verif-paylater-aman",
    "Ancaman Debt Collector": "Denda keterlambatan Rp150.000 telah ditambahkan ke tagihan Paylater Anda. Lunasi sekarang untuk menghindari kunjungan debt collector ke rumah.",
    "Limit Naik (Phishing)": "Selamat! Limit Paylater Anda naik menjadi Rp50.000.000. Klik link ini untuk aktivasi kenaikan limit: www.paylater-limit-naik.com",
    "Diskon Denda Palsu": "Terakhir hari ini! Diskon pelunasan denda 50% jika Anda membayar tagihan tertunggak sekarang. Hubungi CS kami di WA 08123456789.",
    "Konfirmasi Pembayaran": "Pembayaran tagihan listrik bulan ini berhasil diproses. Terima kasih telah menggunakan layanan kami.",
    "Kode OTP": "Kode OTP untuk masuk ke akun Anda adalah 8821. Jangan berikan kode ini kepada siapapun, termasuk pihak yang mengaku dari bank.",
    "Info Paket Kurir": "Halo, paket pesananmu sedang dalam perjalanan bersama kurir. Mohon pastikan ada penerima di alamat tujuan.",
    "Janji Temu": "Selamat siang, apakah hari ini jadi meeting pukul 14.00? Tolong konfirmasinya ya.",
    "Ucapan Terima Kasih": "Terima kasih telah berbelanja di toko kami. Silakan hubungi kami jika ada kendala dengan produk yang diterima."
}

# --- Tampilan UI ---
st.title("🚫 Deteksi Spam Paylater")
st.markdown("Deteksi SMS Spam Paylater menggunakan TF-IDF dan Support Vector Machine")

st.info("💡 **Tips:** Gunakan menu di bawah untuk mencoba berbagai contoh pesan secara cepat.")

# Pilihan Contoh (Dropdown)
pilihan = st.selectbox("📋 Pilih Contoh Pesan:", list(contoh_pesan.keys()))

# Mengambil teks dari pilihan
isi_pesan_awal = contoh_pesan[pilihan]

# Input Teks (Bisa diedit manual)
input_sms = st.text_area(
    "📝 Pesan yang akan dianalisis:",
    value=isi_pesan_awal,
    height=150,
    placeholder="Pilih contoh di atas atau ketik pesan Anda sendiri di sini..."
)

# Tombol Deteksi
if st.button("🔍 Analisis Pesan", type="primary"):
    if input_sms:
        # 1. Preprocessing
        cleaned_text = clean_text(input_sms)

        # 2. Prediksi
        text_vectorized = tfidf.transform([cleaned_text])
        prediction = model.predict(text_vectorized)[0]
        proba = model.predict_proba(text_vectorized)[0]

        # 3. Tampilkan Hasil
        st.divider()
        if prediction == 'spam':
            st.error(f"🚨 **HASIL: TERDETEKSI SPAM!**")
            st.write(f"Keyakinan Model: **{proba[1]*100:.1f}%**")
            st.warning("⚠️ Hati-hati! Pesan ini mengandung indikasi penipuan Paylater atau pencurian data.")
        else:
            st.success(f"✅ **HASIL: PESAN AMAN**")
            st.write(f"Keyakinan Model: **{proba[0]*100:.1f}%**")
            st.info("ℹ️ Pesan ini terlihat normal dan tidak berbahaya.")

    else:
        st.warning("⚠️ Silakan pilih contoh pesan atau ketik sesuatu.")

# Footer
st.markdown("---")
st.caption("Kelompok 1 UAS NLP - Spam SMS Paylater")

Overwriting app.py


#Hasil

In [17]:
# 1. Install Library
!pip install -q pyngrok streamlit

# 2. Masukkan Token Ngrok
from pyngrok import ngrok

# --- TEMPEL KODE TOKEN DARI WEBSITE NGROK DI BAWAH INI ---
token_anda = "37pczPMwbzo3HBkvay9Jv2iRBn8_Cw1zdvQ4sz6WVhBkaXwc"
# Contoh: token_anda = "2AaBbCcDdEeFfGgHhIiJjKkLlMm..."

ngrok.set_auth_token(token_anda)

# 3. Jalankan Aplikasi
import os
# Matikan koneksi lama jika ada
ngrok.kill()

# Jalankan Streamlit di background
os.system("streamlit run app.py --server.port 8501 &")

# Buka koneksi baru
try:
    public_url = ngrok.connect(8501).public_url
    print(f"\n✅ BERHASIL! Klik link ini untuk membuka tugas Anda:\n👉 {public_url}")
except Exception as e:
    print(f"Error: {e}")


✅ BERHASIL! Klik link ini untuk membuka tugas Anda:
👉 https://sulfureously-backswept-lauran.ngrok-free.dev
